In [10]:
import pandas as pd

In [11]:
master = pd.read_csv('/Users/cmcglade/Documents/vaccines/vaccines-github/output_files/sixth_grade_vax.csv')

In [12]:
#clean data
master['enrolled'] = master['enrolled'].str.replace(',', '', regex=True)
master = master[~master['enrolled'].astype(str).str.contains('<')]
master = master[~master['enrolled'].astype(str).str.contains('≤')]
master['enrolled'] = pd.to_numeric(master['enrolled'])

master['dtap'] = master['dtap'].str.replace('%', '', regex=True)
master['polio'] = master['polio'].str.replace('%', '', regex=True)
master['mmr'] = master['mmr'].str.replace('%', '', regex=True)
master['hepb'] = master['hepb'].str.replace('%', '', regex=True)
master['var'] = master['var'].str.replace('%', '', regex=True)

master['dtap'] = pd.to_numeric(master['dtap'])
master['polio'] = pd.to_numeric(master['polio'])
master['mmr'] = pd.to_numeric(master['mmr'])
master['hepb'] = pd.to_numeric(master['hepb'])
master['var'] = pd.to_numeric(master['var'])

master['dtap'] = master['dtap']*.01
master['polio'] = master['polio']*.01
master['mmr'] = master['mmr']*.01
master['hepb'] = master['hepb']*.01
master['var'] = master['var']*.01

master = master.applymap(lambda x: x.lower() if isinstance(x, str) else x)

master['exempt_dtap'] = master['exempt_dtap'].str.replace('%', '', regex=True)
master['exempt_polio'] = master['exempt_polio'].str.replace('%', '', regex=True)
master['exempt_mmr'] = master['exempt_mmr'].str.replace('%', '', regex=True)
master['exempt_hepb'] = master['exempt_hepb'].str.replace('%', '', regex=True)
master['exempt_var'] = master['exempt_var'].str.replace('%', '', regex=True)

master['exempt_dtap'] = pd.to_numeric(master['exempt_dtap'])
master['exempt_polio'] = pd.to_numeric(master['exempt_polio'])
master['exempt_mmr'] = pd.to_numeric(master['exempt_mmr'])
master['exempt_hepb'] = pd.to_numeric(master['exempt_hepb'])
master['exempt_var'] = pd.to_numeric(master['exempt_var'])

master['exempt_dtap'] = master['exempt_dtap']*.01
master['exempt_polio'] = master['exempt_polio']*.01
master['exempt_mmr'] = master['exempt_mmr']*.01
master['exempt_hepb'] = master['exempt_hepb']*.01
master['exempt_var'] = master['exempt_var']*.01

In [13]:
#Calculate raw numbers
import numpy as np
master['mmr_vax_count'] = np.ceil(master['enrolled']*master['mmr'])
master['exempt_count_mmr'] = np.ceil(master['exempt_mmr'] * master['mmr_vax_count'])
master['unvax_mmr'] = np.ceil(master['enrolled'] - master['mmr_vax_count'])
master['missing_forms_mmr'] = master['unvax_mmr']-master['exempt_count_mmr']

master['polio_vax_count'] = np.ceil(master['enrolled']*master['polio'])
master['exempt_count_polio'] = np.ceil(master['exempt_polio'] * master['polio_vax_count'])
master['unvax_polio'] = np.ceil(master['enrolled'] - master['polio_vax_count'])
master['missing_forms_polio'] = master['unvax_polio']-master['exempt_count_polio']

master['dtap_vax_count'] = np.ceil(master['enrolled']*master['dtap'])
master['exempt_count_dtap'] = np.ceil(master['exempt_dtap'] * master['dtap_vax_count'])
master['unvax_dtap'] = np.ceil(master['enrolled'] - master['dtap_vax_count'])
master['missing_forms_dtap'] = master['unvax_dtap']-master['exempt_count_dtap']

master['hep_vax_count'] = np.ceil(master['enrolled']*master['hepb'])
master['exempt_count_hep'] = np.ceil(master['exempt_hepb'] * master['hep_vax_count'])
master['unvax_hep'] = np.ceil(master['enrolled'] - master['hep_vax_count'])
master['missing_forms_hep'] = master['unvax_hep']-master['exempt_count_hep']

master['var_vax_count'] = np.ceil(master['enrolled']*master['var'])
master['exempt_count_var'] = np.ceil(master['exempt_var'] * master['var_vax_count'])
master['unvax_var'] = np.ceil(master['enrolled'] - master['var_vax_count'])
master['missing_forms_var'] = master['unvax_var']-master['exempt_count_var']

In [14]:
#flag 2 or more missing forms
master['mf2_mmr'] = np.where(master['missing_forms_mmr'] >= 2, 'yes', 'no') 
master['mf2_polio'] = np.where(master['missing_forms_polio'] >= 2, 'yes', 'no') 
master['mf2_dtap'] = np.where(master['missing_forms_dtap'] >= 2, 'yes', 'no') 
master['mf2_hep'] = np.where(master['missing_forms_hep'] >= 2, 'yes', 'no')
master['mf2_var'] = np.where(master['missing_forms_var'] >= 2, 'yes', 'no') 

In [15]:
#flag herd immununity
master['mmr_herd_immune'] = np.where(master['mmr'] >= .95, 'yes', 'no') 
master['polio_herd_immune'] = np.where(master['polio'] >= .95, 'yes', 'no') 
master['dtap_herd_immune'] = np.where(master['dtap'] >= .95, 'yes', 'no') 
master['hep_herd_immune'] = np.where(master['hepb'] >= .95, 'yes', 'no') 
master['var_herd_immune'] = np.where(master['var'] >= .95, 'yes', 'no') 

In [16]:
#get total kids per year
totalkids = master.groupby('year')['enrolled'].sum().to_frame(name = 'total_kids').reset_index()

#get total kids vaccinated for mmr each year
measles_vax = master.groupby('year')['mmr_vax_count'].sum().to_frame(name = 'measles_vax_kids').reset_index()

#get total kids vaccinated for polio each year
polio_vax = master.groupby('year')['polio_vax_count'].sum().to_frame(name = 'polio_vax_kids').reset_index()

#get total kids vaccinated for dtap each year
dtap_vax = master.groupby('year')['dtap_vax_count'].sum().to_frame(name = 'dtap_vax_kids').reset_index()

#get total kids vaccinated for hep each year
hep_vax = master.groupby('year')['hep_vax_count'].sum().to_frame(name = 'hep_vax_kids').reset_index()

#get total kids vaccinated for varicella each year
var_vax = master.groupby('year')['var_vax_count'].sum().to_frame(name = 'var_vax_kids').reset_index()

from functools import reduce
# List of DataFrames to merge
data_frames = [totalkids, measles_vax, polio_vax, dtap_vax, hep_vax, var_vax]

# Merge all DataFrames in one line
statewide = reduce(lambda left, right: pd.merge(left, right, on='year'), data_frames)


#calculate percentages
statewide['percent_vax_mmr']= statewide['measles_vax_kids']/statewide['total_kids'] 
statewide['percent_vax_polio']= statewide['polio_vax_kids']/statewide['total_kids'] 
statewide['percent_vax_dtap']= statewide['dtap_vax_kids']/statewide['total_kids'] 
statewide['percent_vax_hep']= statewide['hep_vax_kids']/statewide['total_kids']
statewide['percent_vax_var']= statewide['var_vax_kids']/statewide['total_kids'] 




In [17]:
statewide.to_csv('/Users/cmcglade/Documents/vaccines/vaccines-github/output_files/statewide_vax_rates_6th.csv')

In [18]:
master.to_csv('/Users/cmcglade/Documents/vaccines/vaccines-github/output_files/clean_6th_grade.csv')